# chapter5

In [26]:
# MovieLensのデータセットをdataディレクトリにダウンロードして展開
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

File ‘../data/ml-10m.zip’ already there; not retrieving.

Archive:  ../data/ml-10m.zip


In [27]:
import pandas as pd
pd.set_option('display.max_columns', None)
m_cols = ['movie_id', 'title', 'genre']
display(pd.read_csv('../data/ml-10M100K/movies.dat', sep='::', encoding='latin-1', engine='python'))
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::', encoding='latin-1', engine='python')
movies

,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,2,Jumanji (1995),Adventure|Children|Fantasy
1,3,Grumpier Old Men (1995),Comedy|Romance
2,4,Waiting to Exhale (1995),Comedy|Drama|Romance
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
10675,65088,Bedtime Stories (2008),Adventure|Children|Comedy
10676,65091,Manhattan Melodrama (1934),Crime|Drama|Romance
10677,65126,Choke (2008),Comedy|Drama
10678,65130,Revolutionary Road (2008),Drama|Romance


,movie_id,title,genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10676,65088,Bedtime Stories (2008),Adventure|Children|Comedy
10677,65091,Manhattan Melodrama (1934),Crime|Drama|Romance
10678,65126,Choke (2008),Comedy|Drama
10679,65130,Revolutionary Road (2008),Drama|Romance


In [28]:
movies.genre.apply(lambda x:x.split('|'))

0        [Adventure, Animation, Children, Comedy, Fantasy]
1                           [Adventure, Children, Fantasy]
2                                        [Comedy, Romance]
3                                 [Comedy, Drama, Romance]
4                                                 [Comedy]
                               ...                        
10676                        [Adventure, Children, Comedy]
10677                              [Crime, Drama, Romance]
10678                                      [Comedy, Drama]
10679                                     [Drama, Romance]
10680                                             [Comedy]
Name: genre, Length: 10681, dtype: object

In [29]:
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [30]:
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('/app/data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')
user_tagged_movies

,user_id,movie_id,tag,timestamp
0,15,4973,excellent!,1215184630
1,20,1747,politics,1188263867
2,20,1747,satire,1188263867
3,20,2424,chick flick 212,1188263835
4,20,2424,hanks,1188263835
...,...,...,...,...
95575,71556,1377,Gothic,1188263571
95576,71556,2424,chick flick,1188263606
95577,71556,3033,comedy,1188263626
95578,71556,3081,Gothic,1188263565


In [31]:
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()
user_tagged_movies

,user_id,movie_id,tag,timestamp
0,15,4973,excellent!,1215184630
1,20,1747,politics,1188263867
2,20,1747,satire,1188263867
3,20,2424,chick flick 212,1188263835
4,20,2424,hanks,1188263835
...,...,...,...,...
95575,71556,1377,gothic,1188263571
95576,71556,2424,chick flick,1188263606
95577,71556,3033,comedy,1188263626
95578,71556,3081,gothic,1188263565


In [32]:
print(f'タグ種類={len(user_tagged_movies.tag.unique())}')
print(f'タグレコード数={len(user_tagged_movies)}')
print(f'タグがついている映画数={len(user_tagged_movies.movie_id.unique())}')

タグ種類=15241
タグレコード数=95580
タグがついている映画数=7601


In [33]:
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})
movie_tags

,tag
movie_id,
1,"[pixar, pixar, pixar, animation, pixar, animat..."
2,"[for children, game, animals, joe johnston, ro..."
3,"[funniest movies, comedinha de velhinhos engra..."
4,[girl movie]
5,"[steve martin, pregnancy, remake, steve martin..."
...,...
64993,"[anime, anime, drama, japan, love]"
65006,"[doppleganger, nudity (rear), nudity (topless ..."
65037,"[asperger's syndrome, autism, based on novel, ..."


In [34]:
movies = movies.merge(movie_tags, on='movie_id', how='left')
movies

,movie_id,title,genre,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie]
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin..."
...,...,...,...,...
10676,65088,Bedtime Stories (2008),"[Adventure, Children, Comedy]",NaN
10677,65091,Manhattan Melodrama (1934),"[Crime, Drama, Romance]",NaN
10678,65126,Choke (2008),"[Comedy, Drama]","[chuck palahniuk, based on book]"
10679,65130,Revolutionary Road (2008),"[Drama, Romance]",[toplist08]


In [35]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [36]:
len(ratings)

10000054

In [37]:
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings['user_id'].isin(valid_user_ids)]
ratings

,user_id,movie_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392
...,...,...,...,...
132825,1053,33794,5.0,1134008301
132826,1053,34162,5.0,1134007983
132827,1053,34319,3.5,1134007773
132828,1053,35836,5.0,1134008021


In [38]:
movielens = ratings.merge(movies, on='movie_id')
movielens.head()

,user_id,movie_id,rating,timestamp,title,genre,tag
0,1,122,5.0,838985046,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
1,139,122,3.0,974302621,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
2,149,122,2.5,1112342322,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
3,182,122,3.0,943458784,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
4,215,122,4.5,1102493547,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."


In [39]:
import numpy as np
movielens.groupby('user_id').agg({'movie_id':len}).agg({'movie_id':[min, max, np.mean, len]})

,movie_id
min,20.00
max,1668.00
mean,132.83
len,1000.00


In [40]:
print(f'評価値数={len(movielens)}')

評価値数=132830


In [41]:
movielens.groupby('rating').agg({'movie_id':len})

,movie_id
rating,
0.5,851
1.0,4847
1.5,1247
2.0,10292
2.5,3729
3.0,31706
3.5,9661
4.0,39917
4.5,6949


In [ ]:
movielens.groupby('user_id')['timestamp'].rank(ascending=False, method='first')

0          22.0
1         271.0
2         163.0
3         231.0
4         923.0
          ...  
132825    166.0
132826    114.0
132827    140.0
132828    122.0
132829    203.0
Name: timestamp, Length: 132830, dtype: float64

In [52]:
movielens['timestamp_rank'] = movielens.groupby('user_id')['timestamp'].rank(ascending=False, method='first')
movielens

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
0,1,122,5.0,838985046,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem...",1.0
1,139,122,3.0,974302621,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem...",408.0
2,149,122,2.5,1112342322,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem...",84.0
3,182,122,3.0,943458784,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem...",1104.0
4,215,122,4.5,1102493547,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem...",320.0
...,...,...,...,...,...,...,...,...
132825,1045,57949,0.5,1215617256,"Welcome Home, Roscoe Jenkins (2008)",[Comedy],NaN,76.0
132826,1045,58291,0.5,1215616991,College Road Trip (2008),[Comedy],"[road trip, movie to see]",128.0
132827,1045,59306,3.0,1215617137,Prom Night (2008),"[Horror, Mystery, Thriller]",[remake],102.0
132828,1045,60286,3.0,1215617037,Finding Amanda (2008),"[Comedy, Drama]",NaN,120.0


In [54]:
# みてみる。
movielens.sort_values(['user_id'])

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
0,1,122,5.0,838985046,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem...",1.0
3817,1,588,5.0,838983339,Aladdin (1992),"[Adventure, Animation, Children, Comedy, Musical]","[disney, disney, dvd collection, want to own, ...",22.0
3579,1,586,5.0,838984068,Home Alone (1990),"[Children, Comedy]","[family, christmas, to see, comedy, crime, lat...",10.0
3321,1,539,5.0,838984068,Sleepless in Seattle (1993),"[Comedy, Drama, Romance]","[girlie movie, meg ryan, tom hanks, empire sta...",9.0
3218,1,520,5.0,838984679,Robin Hood: Men in Tights (1993),[Comedy],"[parody, can't remember, very funny!, mel broo...",5.0
...,...,...,...,...,...,...,...,...
53658,1053,2194,4.0,1134007197,"Untouchables, The (1987)","[Action, Crime, Drama]","[gfei own it, gangster, seen more than once, c...",142.0
100451,1053,1257,4.5,1134008016,Better Off Dead... (1985),"[Comedy, Romance]","[directorial debut, nostalgic, skiing, teen, o...",58.0
63714,1053,4571,4.0,1134007848,Bill & Ted's Excellent Adventure (1989),"[Adventure, Comedy, Sci-Fi]","[time travel, dumb but funny, time travel, tee...",77.0
54620,1053,2355,4.0,1134008105,"Bug's Life, A (1998)","[Adventure, Animation, Children, Comedy]","[talking animals, kid flick, pixar, pixar, pix...",39.0


In [55]:
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank'] <= 5]

In [56]:
from typing import List, Dict
from sklearn.metrics import mean_squared_error

class MetricCalculator:
    def calc_rmse(self, true_rating: List[float], pred_rating: List[float]) -> float:
        return np.sqrt(mean_squared_error(true_rating, pred_rating))

    def calc_recall_at_k(self, 
                         true_user2items: Dict[int, List[int]],
                         pred_user2items: Dict[int, List[int]],
                         k: int)-> float:
        scores = []
        for user_id in true_user2items.keys():
            r_at_k = self._recall_at_k(true_user2items[user_id], pred_user2items[user_id], k)
            scores.append(r_at_k)
        return np.mean(scores)

    def _recall_at_k(self, true_items: List[int], pred_items: List[int], k: int) -> float:
        if len(true_items) == 0 or k == 0:
            return 0.0
        r_at_k = (len(set(true_items) & set(pred_items[:k]))) / len(true_items)
        return r_at_k
    
    def calc_precision_at_k(self,
                            true_user2items: Dict[int, List[int]],
                            pred_user2items: Dict[int, List[int]],
                            k: int
                            ) -> float:
        scores = []
        for user_id in true_user2items.keys():
            p_at_k = self._precision_at_k(true_user2items[user_id], pred_user2items[user_id], k)
            scores.append(p_at_k)
        return np.mean(scores)
    
    def _precision_at_k(self,
                        true_items: List[int],
                        pred_items: List[int],
                        k: int
                        ) -> float:
        if k == 0:
            return 0.0
        p_at_k = (len(set(true_items) & set(pred_items[:k]))) / k
        return p_at_k
    
        